<a href="https://colab.research.google.com/github/AdryGP/ABP-FindMo/blob/main/ProbandoDatos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# > ***Procesamiento de datasets con web scraping a IMDB*** 

---





Descarga de los datasets necesarios


In [19]:
!mkdir -p original_datasets/series 

!mkdir -p system_datasets

import gdown as gd

gd.download('https://drive.google.com/uc?export=download&id=1z3skc_GyMiB4j0CzeSfOZGawHheIoJHD', 'original_datasets/series/disney_plus_shows.csv', quiet=False)
gd.download('https://drive.google.com/uc?export=download&id=1-Fy2O1E393Da3jpBu2CCO35FRziqE1BV', 'original_datasets/series/netflix_titles.csv', quiet=False)



Downloading...
From: https://drive.google.com/uc?export=download&id=1z3skc_GyMiB4j0CzeSfOZGawHheIoJHD
To: /content/original_datasets/series/disney_plus_shows.csv
100%|██████████| 418k/418k [00:00<00:00, 56.2MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1-Fy2O1E393Da3jpBu2CCO35FRziqE1BV
To: /content/original_datasets/series/netflix_titles.csv
2.41MB [00:00, 132MB/s]


'original_datasets/series/netflix_titles.csv'

Se carga y se procesa el dataset de descripciones de disney plus con los campos ('id_obra', 'medio', 'nombre', 'descripcion') ya que se dispone de todos los datos en el dataset original


In [20]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import itertools
import time




original_disney_plus=pd.read_csv("original_datasets/series/disney_plus_shows.csv")


original_disney_plus = original_disney_plus[original_disney_plus['type'] == 'series']



original_disney_plus = original_disney_plus.copy()
original_disney_plus.insert(0, 'id_obra', range(0, len(original_disney_plus)))
original_disney_plus = original_disney_plus.assign(id_obra=lambda x: 'S'+x.id_obra.astype(str))



original_disney_plus = original_disney_plus.rename(columns={'title':'nombre', 'plot':'descripcion'})

original_disney_plus = original_disney_plus.assign(medio='S')

original_disney_plus[['id_obra', 'medio', 'nombre', 'descripcion']].to_csv('original_datasets/series/disneyplusDescription.csv', index=False)

disney_plus=pd.read_csv("original_datasets/series/disneyplusDescription.csv")


disney_plus


,id_obra,medio,nombre,descripcion
0,S0,S,101 Dalmatian Street,This series follows the lives of Delilah and D...
1,S1,S,America's Funniest Home Videos,Viewers from around America send in home video...
2,S2,S,America's National Parks,Series exploring the incredible wildlife of Am...
3,S3,S,Andi Mack,A contemporary coming-of-age story about a gir...
4,S4,S,Austin & Ally,"Following the lives of Austin, an aspiring con..."
...,...,...,...,...
186,S186,S,Wizards of Waverly Place,The Russo family may be an ordinary family wit...
187,S187,S,Wolverine and the X-Men,"After an explosion at the school, the X-Men we..."
188,S188,S,BarkFest,Nat Geo WILD celebrates pawsome dogs with its ...
189,S189,S,X-Men: Evolution,"This rendition of X-Men features Cyclops, Jean..."


Funciones usadas en el web scraping para IMDB

In [21]:
##############################
#  Module: imdbUtils.py
#  Author: Shravan Kuchkula
#  Date: 07/13/2019
##############################

import requests
from bs4 import BeautifulSoup

def getSoup(url):
    """
    Utility function which takes a url and returns a Soup object.
    """
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    return soup

def minMax(a):
    '''Returns the index of negative and positive review.'''
    
    # get the index of least rated user review
    minpos = a.index(min(a))
    
    # get the index of highest rated user review
    maxpos = a.index(max(a))
    
    return minpos, maxpos

def getReviews(soup):
    '''Function returns a negative and positive review for each movie.'''
    
    # get a list of user ratings
    user_review_ratings = [tag.previous_element for tag in 
                           soup.find_all('span', attrs={'class': 'point-scale'})]
    
    
    # find the index of negative and positive review
    n_index, p_index = minMax(list(map(int, user_review_ratings)))
    
    
    # get the review tags
    user_review_list = soup.find_all('a', attrs={'class':'title'})
    
    
    # get the negative and positive review tags
    n_review_tag = user_review_list[n_index]
    p_review_tag = user_review_list[p_index]
    
    # return the negative and positive review link
    n_review_link = "https://www.imdb.com" + n_review_tag['href']
    p_review_link = "https://www.imdb.com" + p_review_tag['href']
    
    return n_review_link, p_review_link

def getReviewText(review_url):
    '''Returns the user review text given the review url.'''
    
    # get the review_url's soup
    soup = getSoup(review_url)
    
    # find div tags with class text show-more__control
    tag = soup.find('div', attrs={'class': 'text show-more__control'})
    
    return tag.getText()


def getQualifications(review_url):
    '''Returns the user review text given the review url.'''
    
    # get the review_url's soup
    soup = getSoup(review_url)
    
    # find div tags with class text show-more__control
    tag = soup.find('span', attrs={'class': 'rating-other-user-rating'})
    
    return tag.getText()  


def getMovieTitle(review_url):
    '''Returns the movie title from the review url.'''
    
    # get the review_url's soup
    soup = getSoup(review_url)
    
    # find h1 tag
    tag = soup.find('h1')
    
    return list(tag.children)[1].getText()

def getNounChunks(user_review):
    
    # create the doc object
    doc = nlp(user_review)
    
    # get a list of noun_chunks
    noun_chunks = list(doc.noun_chunks)
    
    # convert noun_chunks from span objects to strings, otherwise it won't pickle
    noun_chunks_strlist = [chunk.text for chunk in noun_chunks]
    
    return noun_chunks_strlist

Como se necesitan las reviews se obtiene la dirección de cada review de todas las series del dataset

In [22]:
base_url = "https://www.imdb.com"
original_disney_plus = original_disney_plus.copy()

original_disney_plus.insert(0,'reviews',base_url + '/title/' + original_disney_plus['imdb_id'] + '/reviews')

original_disney_plus

,reviews,id_obra,imdb_id,nombre,descripcion,type,rated,year,released_at,added_at,runtime,genre,director,writer,actors,language,country,awards,metascore,imdb_rating,imdb_votes,medio
1,https://www.imdb.com/title/tt7019028/reviews,S0,tt7019028,101 Dalmatian Street,This series follows the lives of Delilah and D...,series,NaN,2018–,25 Mar 2019,"February 28, 2020",NaN,"Animation, Comedy, Family",NaN,NaN,"Josh Brener, Michaela Dietz, Bert Davis, Abiga...",English,"UK, USA, Canada",NaN,NaN,6.2,124,S
25,https://www.imdb.com/title/tt0098740/reviews,S1,tt0098740,America's Funniest Home Videos,Viewers from around America send in home video...,series,TV-PG,1989–,26 Nov 1989,"April 24, 2020",30 min,"Comedy, Family, Reality-TV",NaN,NaN,"Jess Harnell, Tom Bergeron, Bob Saget, Ernie A...",English,USA,4 wins & 6 nominations.,NaN,6.2,"4,652",S
27,https://www.imdb.com/title/tt4622802/reviews,S2,tt4622802,America's National Parks,Series exploring the incredible wildlife of Am...,series,TV-G,2015–,06 Dec 2015,"January 15, 2020",NaN,Documentary,NaN,NaN,NaN,English,USA,NaN,NaN,7.9,73,S
30,https://www.imdb.com/title/tt5195114/reviews,S3,tt5195114,Andi Mack,A contemporary coming-of-age story about a gir...,series,TV-G,2017–2019,10 Mar 2017,"November 12, 2019",25 min,"Comedy, Family",NaN,Terri Minsky,"Peyton Elizabeth Lee, Joshua Rush, Sofia Wylie...",English,USA,3 wins & 6 nominations.,NaN,7.3,"1,723",S
39,https://www.imdb.com/title/tt1830491/reviews,S4,tt1830491,Austin & Ally,"Following the lives of Austin, an aspiring con...",series,TV-G,2011–2016,02 Dec 2011,"January 1, 2020",30 min,"Comedy, Family, Music",NaN,"Kevin Kopelow, Heath Seifert","Ross Lynch, Laura Marano, Raini Rodriguez, Cal...",English,USA,10 wins & 14 nominations.,NaN,5.9,"8,666",S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,https://www.imdb.com/title/tt0799922/reviews,S186,tt0799922,Wizards of Waverly Place,The Russo family may be an ordinary family wit...,series,TV-G,2007–2012,12 Oct 2007,"November 12, 2019",22 min,"Adventure, Comedy, Drama, Family, Fantasy",NaN,Todd J. Greenwald,"Selena Gomez, David Henrie, Jake T. Austin, Je...",English,USA,Won 2 Primetime Emmys. Another 35 wins & 50 no...,NaN,6.8,"27,826",S
976,https://www.imdb.com/title/tt0772145/reviews,S187,tt0772145,Wolverine and the X-Men,"After an explosion at the school, the X-Men we...",series,TV-Y7,2008–2009,23 Jan 2009,"November 12, 2019",23 min,"Animation, Action, Adventure, Drama, Sci-Fi",NaN,NaN,"Steve Blum, Jim Ward, Nolan North, Fred Tatasc...",English,USA,NaN,NaN,8.0,"9,497",S
977,https://www.imdb.com/title/tt6959706/reviews,S188,tt6959706,BarkFest,Nat Geo WILD celebrates pawsome dogs with its ...,series,NaN,2017–,NaN,"November 12, 2019",35 min,NaN,NaN,NaN,NaN,English,USA,NaN,NaN,NaN,NaN,S
979,https://www.imdb.com/title/tt0247827/reviews,S189,tt0247827,X-Men: Evolution,"This rendition of X-Men features Cyclops, Jean...",series,TV-Y7,2000–2003,04 Nov 2000,"November 12, 2019",23 min,"Animation, Action, Drama, Fantasy, Romance, Sc...",NaN,NaN,"Kirby Morrow, Venus Terzo, David Kaye, Brad Sw...",English,USA,3 wins & 6 nominations.,NaN,7.9,"9,763",S


Se transforman las urls del dataset a una lista para que sean usables por las funciones de web scraping

In [ ]:
movie_links=original_disney_plus['reviews'].tolist()

Se obtiene la dirección de cada review individual

In [24]:
movie_soups =[]
for val in movie_links:
  try:
    var = getReviews(getSoup(val))
    movie_soups.append(var)
  except:
    pass
    


In [25]:
movie_soups = list(itertools.chain(*movie_soups))


Se obtiene el texto de cada review

In [26]:
review_texts =[]
for val in movie_soups:
  
  try:
    var = getReviewText(val)
    review_texts.append(var)
  except:
    pass

Se obtiene el título de la obra al que corresponde dicha review

In [27]:
titles =[]
for val in movie_soups:
  
  try:
    var = getMovieTitle(val)
    titles.append(var)
  except:
    pass

Se obtiene la calificación de cada review

In [28]:
qualifications =[]
for val in movie_soups:
  
  try:
    var = getQualifications(val)
    qualifications.append(var)
  except:
    qualifications.append('null/')
    pass

Se usa el criterio que hemos estipulado de +/-/~ dependiendo de la nota de cada review ya que sus notas oscilan de 0/10 en su origen

In [29]:
sentiment=[]
for val in qualifications:
  val.replace('\\n','')
  val=val.strip()
  var=val.find('/')
  val=val[0:var]
  if val!='null':
    if int(val)>5:
      sentiment.append("+")
    elif int(val)==5:
      sentiment.append("~")
    else:
      sentiment.append("-")
  else:
    sentiment.append('null')



Se crea el dataframe con los datos necesarios para el csv de valoraciones( 'id_obra', 'comentario', 'valoracion') el id se obtiene a partir del dataset de descripciones usando un join de los mismos

In [30]:
df = pd.DataFrame({'nombre': titles,
             'comentario': review_texts,'valoracion': sentiment})


frames=[df,disney_plus]

dff=pd.merge(df,disney_plus,how='inner',on=['nombre'])


dff[['id_obra', 'comentario', 'valoracion',]].to_csv('original_datasets/series/disneyplusValoracion.csv', index=False)

dff=pd.read_csv('original_datasets/series/disneyplusValoracion.csv')




Se obtiene el dataset de descripciones de netflix con los campos estipulados('id_obra', 'medio', 'nombre', 'descripcion')

In [31]:
netflix=pd.read_csv('original_datasets/series/netflix_titles.csv')
netflix = netflix[netflix['type'] == 'TV Show']

netflix = netflix.copy()
netflix.insert(0, 'id_obra', range(0, len(netflix)))
netflix = netflix.assign(id_obra=lambda x: 'S'+x.id_obra.astype(str))

netflix = netflix.rename(columns={'title':'nombre', 'description':'descripcion'})

netflix = netflix.assign(medio='S')

netflix[['id_obra', 'medio', 'nombre', 'descripcion']].to_csv('original_datasets/series/netflixDescription.csv', index=False)

netflix=pd.read_csv('original_datasets/series/netflixDescription.csv')
netflix

,id_obra,medio,nombre,descripcion
0,S0,S,Transformers Prime,"With the help of three human allies, the Autob..."
1,S1,S,Transformers: Robots in Disguise,When a prison ship crash unleashes hundreds of...
2,S2,S,Apaches,A young journalist is forced into a life of cr...
3,S3,S,Fire Chasers,"As California's 2016 fire season rages, brave ..."
4,S4,S,Castle of Stars,As four couples with different lifestyles go t...
...,...,...,...,...
1964,S1964,S,Kikoriki,A wacky rabbit and his gang of animal pals hav...
1965,S1965,S,Red vs. Blue,"This parody of first-person shooter games, mil..."
1966,S1966,S,Maron,"Marc Maron stars as Marc Maron, who interviews..."
1967,S1967,S,A Young Doctor's Notebook and Other Stories,"Set during the Russian Revolution, this comic ..."


Se instala un librería de web scraping para IMDB a mayores para poder obtener los links base de las obras

In [32]:
!pip install imdbpy

     |████████████████████████████████| 307kB 2.7MB/s 


Se obtienen las URL de cada obra

In [33]:
import imdb


titles=netflix['nombre'].tolist()
urls=[]

for show in titles:
  movie = show
  ia = imdb.IMDb()
  results = ia.search_movie(movie)
  try:
    mv = results[0] #First result
    URL = ia.get_imdbURL(mv) 
    urls.append(URL + 'reviews')
  except:
    pass


Se guardan las URL en un fichero por si fuere necesario su uso

In [34]:
file = open("original_datasets/series/urls.txt", "w")
for url in urls:
  file.write(url+',')
file.close()

file = open("original_datasets/series/urls.txt", "r")

urlaux=file.read()

urls=list(urlaux.split(","))


Se obtienen los links de las reviews de cada obra

In [35]:
reviewsLink =[]
for val in urls:
  try:
    var = getReviews(getSoup(val))
    reviewsLink.append(var)
  except:
    pass
    


In [36]:
reviewsLink = list(itertools.chain(*reviewsLink))

Se obtiene el contenido de cada review

In [37]:
review_texts_netflix =[]
for val in reviewsLink:
  try:
    var = getReviewText(val)
    review_texts_netflix.append(var)
  except:
    pass

Se obtiene el título de la obra a la que pertenece cada review

In [38]:
titles_netflix =[]
for val in reviewsLink:
  try:
    var = getMovieTitle(val)
    titles_netflix.append(var)
  except:
    pass

Se obtienen las calificaciones de cada obra

In [39]:
qualifications_netflix =[]
for val in reviewsLink: 
  try:
    var = getQualifications(val)
    qualifications_netflix.append(var)
  except:
    qualifications_netflix.append('null/')
    pass

Se traducen las valoraciones a la representación ya mencionada dependiendo de la nota

In [40]:
sentiment_netflix=[]
for val in qualifications_netflix:
  val.replace('\\n','')
  val=val.strip()
  var=val.find('/')
  val=val[0:var]
  if val!='null':
    if int(val)>5:
      sentiment_netflix.append("+")
    elif int(val)==5:
      sentiment_netflix.append("~")
    else:
      sentiment_netflix.append("-")
  else:
    sentiment_netflix.append('null')




Se genera el dataframe de las valoraciones con los datos obtenidos de la misma forma que se ha hecho con el de disney plus

In [41]:
df = pd.DataFrame({'nombre': titles_netflix,
             'comentario': review_texts_netflix,'valoracion': sentiment_netflix})


frames=[df,netflix]

dff=pd.merge(df,netflix,how='inner',on=['nombre'])



dff[['id_obra', 'comentario', 'valoracion',]].to_csv('original_datasets/series/netflixValoracion.csv', index=False)

dff=pd.read_csv('original_datasets/series/netflixValoracion.csv')

dff

,id_obra,comentario,valoracion
0,S0,"I know what i'm talking about when I say, THIS...",+
1,S0,I find this series to be a great take on the T...,+
2,S1,Transformer Prime was one of the best shows I'...,-
3,S1,"I have to say,this show is way better than the...",+
4,S2,Another outstanding series from Netflix. Excel...,+
...,...,...,...
2439,S1965,My brother's friend had a DVD laying around ne...,NaN
2440,S1966,Marc Maron has been on the stand up circuit fo...,-
2441,S1966,"To be honest, I never heard of Marc Maron befo...",+
2442,S1968,Everyone says that Seinfield is the greatest s...,NaN
